In [1]:
# Chances of Recovery

# Age is inversely proportional to Recovery.

# Chances of Recovery = Recovered Count / Death Count * (1 / (Country's Median Age/100))

SyntaxError: invalid syntax (<ipython-input-1-747e8e7d1315>, line 5)

In [18]:
import pandas as pd
import json
# df = pd.read_csv('../covid19/data/median_age.tsv', delimiter="|", header=0)

data = {}
for row in df.iterrows():
    v = row[1].values
    data[v[1].strip()] =v[2]

json.dump(data, open("../covid19/data/median_age.json", 'w'), indent=2)

